# utils

In [1]:
import vectorbt as vbt

from vectorbt.utils import checks, config, decorators, attr

In [2]:
import numpy as np
import pandas as pd
from numba import njit

In [3]:
v1 = 0
a1 = np.array([1])
a2 = np.array([1, 2, 3])
a3 = np.array([[1, 2, 3]])
a4 = np.array([[1], [2], [3]])
a5 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
sr1 = pd.Series([1], index=pd.Index(['x1'], name='i1'))
print(sr1)
sr2 = pd.Series([1, 2, 3], index=pd.Index(['x2', 'y2', 'z2'], name='i2'))
print(sr2)
df1 = pd.DataFrame(
    [[1]], 
    index=pd.Index(['x3'], name='i3'), 
    columns=pd.Index(['a3'], name='c3'))
print(df1)
df2 = pd.DataFrame(
    [[1], [2], [3]], 
    index=pd.Index(['x4', 'y4', 'z4'], name='i4'), 
    columns=pd.Index(['a4'], name='c4'))
print(df2)
df3 = pd.DataFrame(
    [[1, 2, 3]], 
    index=pd.Index(['x5'], name='i5'), 
    columns=pd.Index(['a5', 'b5', 'c5'], name='c5'))
print(df3)
df4 = pd.DataFrame(
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]], 
    index=pd.Index(['x6', 'y6', 'z6'], name='i6'), 
    columns=pd.Index(['a6', 'b6', 'c6'], name='c6'))
print(df4)

multi_i = pd.MultiIndex.from_arrays([['x7', 'y7', 'z7'], ['x8', 'y8', 'z8']], names=['i7', 'i8']) 
multi_c = pd.MultiIndex.from_arrays([['a7', 'b7', 'c7'], ['a8', 'b8', 'c8']], names=['c7', 'c8'])
df5 = pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], index=multi_i, columns=multi_c)
print(df5)

i1
x1    1
dtype: int64
i2
x2    1
y2    2
z2    3
dtype: int64
c3  a3
i3    
x3   1
c4  a4
i4    
x4   1
y4   2
z4   3
c5  a5  b5  c5
i5            
x5   1   2   3
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  2  3
y7 y8  4  5  6
z7 z8  7  8  9


## config

In [4]:
conf = config.Config({'a': 0, 'b': {'c': 1}}, frozen_keys=False)
conf['b']['d'] = 2

conf = config.Config({'a': 0, 'b': {'c': 1}}, frozen_keys=True)
conf['a'] = 2

try:
    conf['d'] = 2
except Exception as e:
    print(e)

try:
    conf.update(d=2)
except Exception as e:
    print(e)
    
conf.update(d=2, force=True)

"Config keys are frozen: key 'd' not found"
"Config keys are frozen: key 'd' not found"


In [5]:
conf = config.Config({'a': 0, 'b': {'c': 1}}, readonly=True)

try:
    conf['a'] = 2
except Exception as e:
    print(e)
    
try:
    del conf['a']
except Exception as e:
    print(e)
    
try:
    conf.pop('a')
except Exception as e:
    print(e)
    
try:
    conf.popitem()
except Exception as e:
    print(e)
    
try:
    conf.clear()
except Exception as e:
    print(e)
    
try:
    conf.update(a=2)
except Exception as e:
    print(e)
    
print(conf.merge_with(dict(b=dict(d=2))))

Config is read-only
Config is read-only
Config is read-only
Config is read-only
Config is read-only
Config is read-only
{'a': 0, 'b': {'d': 2}}


In [6]:
print(config.merge_dicts({'a': 1}, {'b': 2}))
print(config.merge_dicts({'a': 1}, {'a': 2}))
print(config.merge_dicts({'a': {'b': 2}}, {'a': {'c': 3}}))
print(config.merge_dicts({'a': {'b': 2}}, {'a': {'b': 3}}))

{'a': 1, 'b': 2}
{'a': 2}
{'a': {'b': 2, 'c': 3}}
{'a': {'b': 3}}


In [7]:
class H(config.Configured):
    def __init__(self, a, b=2, **kwargs):
        super().__init__(a=a, b=b, **kwargs)
        
print(H(1).config)
print(H(1).copy(b=3).config)
print(H(1).copy(c=4).config)

Config({
    "a": 1,
    "b": 2
})
Config({
    "a": 1,
    "b": 3
})
Config({
    "a": 1,
    "b": 2,
    "c": 4
})


## decorators

In [8]:
class G():
    @decorators.class_or_instancemethod
    def g(self_or_cls):
        if isinstance(self_or_cls, type):
            print("class")
        else:
            print("instance")
            
G.g()
G().g()

class
instance


In [9]:
class G():
    @decorators.cached_property(hello="world", hello2="world2")
    def cache_me(self): return np.random.uniform(size=(10000, 10000))
    
G.cache_me.flags

{'hello': 'world', 'hello2': 'world2'}

In [10]:
g = G()

In [11]:
%time _ = g.cache_me
%time _ = g.cache_me

CPU times: user 1.19 s, sys: 128 ms, total: 1.32 s
Wall time: 1.32 s
CPU times: user 24 µs, sys: 0 ns, total: 24 µs
Wall time: 26 µs


In [12]:
dir(g)

['__cached_cache_me',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cache_me']

In [13]:
G.cache_me.clear_cache(g)
%time _ = g.cache_me
%time _ = g.cache_me

CPU times: user 1.2 s, sys: 165 ms, total: 1.37 s
Wall time: 1.44 s
CPU times: user 41 µs, sys: 162 µs, total: 203 µs
Wall time: 204 µs


In [14]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(instance=g, func='cache_me'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.19 s, sys: 168 ms, total: 1.36 s
Wall time: 1.36 s
CPU times: user 1.2 s, sys: 176 ms, total: 1.37 s
Wall time: 1.37 s


In [15]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(func='cache_me'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.2 s, sys: 180 ms, total: 1.38 s
Wall time: 1.38 s
CPU times: user 1.19 s, sys: 168 ms, total: 1.36 s
Wall time: 1.36 s


In [16]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(instance=g))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.18 s, sys: 165 ms, total: 1.35 s
Wall time: 1.35 s
CPU times: user 1.19 s, sys: 172 ms, total: 1.36 s
Wall time: 1.36 s


In [17]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(cls=G))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.18 s, sys: 164 ms, total: 1.35 s
Wall time: 1.35 s
CPU times: user 1.18 s, sys: 161 ms, total: 1.35 s
Wall time: 1.34 s


In [18]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(cls='G'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.17 s, sys: 162 ms, total: 1.33 s
Wall time: 1.33 s
CPU times: user 1.17 s, sys: 158 ms, total: 1.33 s
Wall time: 1.33 s


In [19]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(cls='g'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.17 s, sys: 161 ms, total: 1.34 s
Wall time: 1.33 s
CPU times: user 23 µs, sys: 1 µs, total: 24 µs
Wall time: 26.2 µs


In [20]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(flags={'hello': 'world'}))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.18 s, sys: 161 ms, total: 1.34 s
Wall time: 1.34 s
CPU times: user 1.17 s, sys: 159 ms, total: 1.33 s
Wall time: 1.33 s


In [21]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(flags={'hello': 'world', 'hello2': 'world2'}))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.18 s, sys: 161 ms, total: 1.34 s
Wall time: 1.34 s
CPU times: user 1.18 s, sys: 161 ms, total: 1.34 s
Wall time: 1.34 s


In [22]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(flags={'hello': 'world', 'hello2': 'world2', 'hello3': 'world3'}))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.19 s, sys: 163 ms, total: 1.36 s
Wall time: 1.36 s
CPU times: user 26 µs, sys: 0 ns, total: 26 µs
Wall time: 26.9 µs


In [23]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.17 s, sys: 163 ms, total: 1.33 s
Wall time: 1.33 s
CPU times: user 1.17 s, sys: 169 ms, total: 1.34 s
Wall time: 1.34 s


In [24]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(instance=g, func='cache_me'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.17 s, sys: 167 ms, total: 1.34 s
Wall time: 1.34 s
CPU times: user 28 µs, sys: 0 ns, total: 28 µs
Wall time: 30 µs


In [25]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(func='cache_me'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.18 s, sys: 162 ms, total: 1.34 s
Wall time: 1.34 s
CPU times: user 28 µs, sys: 1 µs, total: 29 µs
Wall time: 29.1 µs


In [26]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(instance=g))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.19 s, sys: 163 ms, total: 1.35 s
Wall time: 1.35 s
CPU times: user 27 µs, sys: 0 ns, total: 27 µs
Wall time: 27.9 µs


In [27]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(cls=G))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.17 s, sys: 167 ms, total: 1.34 s
Wall time: 1.34 s
CPU times: user 25 µs, sys: 0 ns, total: 25 µs
Wall time: 26 µs


In [28]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(cls='G'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.19 s, sys: 164 ms, total: 1.35 s
Wall time: 1.35 s
CPU times: user 28 µs, sys: 0 ns, total: 28 µs
Wall time: 29.1 µs


In [29]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(flags={'hello': 'world'}))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.19 s, sys: 166 ms, total: 1.36 s
Wall time: 1.36 s
CPU times: user 26 µs, sys: 0 ns, total: 26 µs
Wall time: 27.9 µs


In [30]:
class G():
    @decorators.cached_method(hello="world", hello2="world2")
    def cache_me(self, a): return np.random.uniform(size=(10000, 10000)) * a

G.cache_me.flags

{'hello': 'world', 'hello2': 'world2'}

In [31]:
g = G()

In [32]:
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)

CPU times: user 1.21 s, sys: 193 ms, total: 1.4 s
Wall time: 1.41 s
CPU times: user 31 µs, sys: 1 µs, total: 32 µs
Wall time: 31.9 µs


In [33]:
dir(g)

['__cached_cache_me',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cache_me']

In [34]:
G.cache_me.clear_cache(g)
%time _ = g.cache_me(2)

CPU times: user 1.23 s, sys: 188 ms, total: 1.41 s
Wall time: 1.41 s


In [35]:
G.cache_me.clear_cache(g)
G.cache_me.disabled = True
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
G.cache_me.disabled = False

CPU times: user 1.23 s, sys: 189 ms, total: 1.42 s
Wall time: 1.41 s
CPU times: user 31 µs, sys: 1 µs, total: 32 µs
Wall time: 33.1 µs


In [36]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(func=g.cache_me))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 183 ms, total: 1.41 s
Wall time: 1.41 s
CPU times: user 1.23 s, sys: 190 ms, total: 1.42 s
Wall time: 1.42 s


In [37]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(instance=g, func='cache_me'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.21 s, sys: 187 ms, total: 1.4 s
Wall time: 1.4 s
CPU times: user 1.22 s, sys: 184 ms, total: 1.41 s
Wall time: 1.41 s


In [38]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(func='cache_me'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 185 ms, total: 1.41 s
Wall time: 1.41 s
CPU times: user 1.23 s, sys: 181 ms, total: 1.41 s
Wall time: 1.41 s


In [39]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(instance=g))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 186 ms, total: 1.41 s
Wall time: 1.41 s
CPU times: user 1.23 s, sys: 189 ms, total: 1.41 s
Wall time: 1.41 s


In [40]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(cls=G))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 188 ms, total: 1.41 s
Wall time: 1.41 s
CPU times: user 1.23 s, sys: 183 ms, total: 1.41 s
Wall time: 1.41 s


In [41]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(cls='G'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 191 ms, total: 1.42 s
Wall time: 1.42 s
CPU times: user 1.23 s, sys: 182 ms, total: 1.41 s
Wall time: 1.41 s


In [42]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(cls='g'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.22 s, sys: 185 ms, total: 1.41 s
Wall time: 1.41 s
CPU times: user 34 µs, sys: 1e+03 ns, total: 35 µs
Wall time: 35 µs


In [43]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(flags={'hello': 'world'}))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.21 s, sys: 186 ms, total: 1.4 s
Wall time: 1.4 s
CPU times: user 1.24 s, sys: 221 ms, total: 1.46 s
Wall time: 1.46 s


In [44]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(flags={'hello': 'world', 'hello2': 'world2'}))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.21 s, sys: 191 ms, total: 1.4 s
Wall time: 1.4 s
CPU times: user 1.23 s, sys: 203 ms, total: 1.43 s
Wall time: 1.43 s


In [45]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(vbt.CacheCondition(flags={'hello': 'world', 'hello2': 'world2', 'hello3': 'world3'}))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 200 ms, total: 1.43 s
Wall time: 1.43 s
CPU times: user 34 µs, sys: 0 ns, total: 34 µs
Wall time: 35 µs


In [46]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 196 ms, total: 1.42 s
Wall time: 1.42 s
CPU times: user 1.23 s, sys: 185 ms, total: 1.41 s
Wall time: 1.41 s


In [47]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(instance=g, func='cache_me'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 201 ms, total: 1.43 s
Wall time: 1.43 s
CPU times: user 37 µs, sys: 1e+03 ns, total: 38 µs
Wall time: 37.9 µs


In [48]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(func='cache_me'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.24 s, sys: 248 ms, total: 1.49 s
Wall time: 1.49 s
CPU times: user 37 µs, sys: 1 µs, total: 38 µs
Wall time: 38.9 µs


In [49]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(instance=g))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.26 s, sys: 276 ms, total: 1.53 s
Wall time: 1.53 s
CPU times: user 36 µs, sys: 1 µs, total: 37 µs
Wall time: 37.2 µs


In [50]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(cls=G))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 211 ms, total: 1.44 s
Wall time: 1.44 s
CPU times: user 36 µs, sys: 0 ns, total: 36 µs
Wall time: 37.7 µs


In [51]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(cls='G'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 206 ms, total: 1.44 s
Wall time: 1.43 s
CPU times: user 36 µs, sys: 0 ns, total: 36 µs
Wall time: 37.9 µs


In [52]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(vbt.CacheCondition(flags={'hello': 'world'}))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 193 ms, total: 1.42 s
Wall time: 1.42 s
CPU times: user 34 µs, sys: 1e+03 ns, total: 35 µs
Wall time: 34.8 µs


In [53]:
# Non-hashable arguments won't cache
%time _ = g.cache_me(np.asarray(2))
%time _ = g.cache_me(np.asarray(2))

CPU times: user 1.23 s, sys: 161 ms, total: 1.39 s
Wall time: 1.39 s
CPU times: user 1.27 s, sys: 302 ms, total: 1.57 s
Wall time: 1.78 s


## checks

In [54]:
print(checks.is_series(v1))
print(checks.is_series(a1))
print(checks.is_series(sr1))
print(checks.is_series(df1))

False
False
True
False


In [55]:
print(checks.is_frame(v1))
print(checks.is_frame(a1))
print(checks.is_frame(sr1))
print(checks.is_frame(df1))

False
False
False
True


In [56]:
print(checks.is_pandas(v1))
print(checks.is_pandas(a1))
print(checks.is_pandas(sr1))
print(checks.is_pandas(df1))

False
False
True
True


In [57]:
print(checks.is_any_array(v1))
print(checks.is_any_array(a1))
print(checks.is_any_array(sr1))
print(checks.is_any_array(df1))

False
True
True
True


In [58]:
print(checks.is_numba_func(lambda x: x))
print(checks.is_numba_func(njit(lambda x: x)))

False
True


In [59]:
print(checks.is_hashable(2))
print(checks.is_hashable(np.asarray(2)))

True
False


In [60]:
checks.assert_in(0, (0, 1))

In [61]:
checks.assert_numba_func(njit(lambda x: x))

In [62]:
checks.assert_not_none(v1)

In [63]:
checks.assert_type(v1, int)
checks.assert_type(a1, np.ndarray)
checks.assert_type(sr1, (np.ndarray, pd.Series))

In [64]:
checks.assert_type_equal(v1, v1)
checks.assert_type_equal(a1, a2)
checks.assert_type_equal(sr1, sr1)
checks.assert_type_equal(df1, df2)

In [65]:
checks.assert_dtype(a1, np.integer)

In [66]:
checks.assert_dtype_equal(v1, a1)
checks.assert_dtype_equal(a1, df1)
checks.assert_dtype_equal(df1, df2)
checks.assert_dtype_equal(df2, df3)

In [67]:
checks.assert_ndim(v1, 0)
checks.assert_ndim(a1, 1)
checks.assert_ndim(df1, 2)

In [68]:
checks.assert_len_equal([[1]], [[2]])

In [69]:
checks.assert_shape_equal(a1, sr1)
checks.assert_shape_equal(df2, df4, axis=0)
checks.assert_shape_equal(df3, df4, axis=1)
checks.assert_shape_equal(df2, df3, axis=(0, 1))

In [70]:
checks.assert_index_equal(df3.index, df3.index)

In [71]:
checks.assert_meta_equal(df3, df3)

In [72]:
checks.assert_array_equal(df3, df3)

In [73]:
checks.assert_level_not_exists(df3.columns, 'a')